In [1]:
from model.diffusion import Trainer
from model.unet import UNet
from model.fdnunet import FDNUNet

/home/namanwsl/code/fea-diffusion/model/diffusion.py:3: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
model = UNet(
    input_dim=64,
    num_channels=2, # geometry/displacement (2)
    num_condition_channels=4, # constraints (1) + force (2)
)

In [3]:
model = FDNUNet(
    input_dim=64,
    num_channels=2, # geometry/displacement (2)
    num_condition_channels=1, # geometry (1)
    num_auxiliary_condition_channels=3, # constraints (1) + force (2)
    num_stages=4
)

In [4]:
# Number of parameters
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

57086594


In [5]:
print(model)

FDNUNet(
  (time_embedding_mlp): Sequential(
    (0): SinusoidalPosEmb()
    (1): Linear(in_features=64, out_features=256, bias=True)
    (2): GELU(approximate='none')
    (3): Linear(in_features=256, out_features=256, bias=True)
  )
  (condition_feature_extractor): ConditionFeatureExtractor(
    (pre_extractors): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): SiLU()
      (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): SiLU()
      (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): SiLU()
      (6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): SiLU()
    )
    (extractors): ModuleList(
      (0): Sequential(
        (0): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): SiLU()
      )
      (1): Sequential(
        (0): Downsample(
          (downsample): Sequential(
            (0): Rearrange('b c (h p1) 

In [6]:
trainer = Trainer(
    model=model,
    dataset_folder='data/feadata2500',
    use_dataset_augmentation=True,
    sample_dataset_folder='data/sample_1/',
    num_sample_conditions_per_plate=1,
    dataset_image_size=128,
    train_batch_size=1,
    train_learning_rate=1e-4,
    num_train_steps=1,
    num_gradient_accumulation_steps=16,
    num_steps_per_milestone=1,
    ema_steps_per_milestone=1,
    results_folder='results',
    use_batch_split_over_devices=True,
)
# trainer = Trainer(
#     model=model,
#     dataset_folder='data/feadata2500',
#     use_dataset_augmentation=True,
#     sample_dataset_folder='data/sample_1/',
#     num_sample_conditions_per_plate=1,
#     dataset_image_size=256,
#     train_batch_size=8,
#     train_learning_rate=1e-4,
#     num_train_steps=16,
#     num_gradient_accumulation_steps=2,
#     num_steps_per_milestone=1,
#     ema_steps_per_milestone=1,
#     results_folder='results',
#     use_batch_split_over_devices=True,
# )
# each effective batch (number of samples that are used for updating the loss once) is train_batch_size * num_gradient_accumulation_steps
# hence to go over the entire dataset once, we need len(dataset) / (train_batch_size * num_gradient_accumulation_steps) steps = len(train_dataloader) / num_gradient_accumulation_steps steps

In [7]:
trainer.train()

Epoch Size: 6250.0 effective batches
Number of Effective Epochs: 0.00016


loss: 116139.6077 : 100%|██████████| 1/1 [00:48<00:00, 48.12s/it]


Training done!
